# Consumption Based Asset Pricing through GMM

## Empirical Asset Pricing
## Lecturer: Juan F. Imbet
## Paris-Dauphine University

The first order condition of an investor maximizing expected utility and smoothing consumption by investing in a financial asset satisfies

$$ 
1 = \mathbb{E}_t[M_{t+1} R_{t+1}]
$$

The simplest parametrization of the Stochastic Discount Factor (SDF) that assumes power utility (CRRA) implies


$$
1 = \mathbb{E}_t \Big[\delta \Big(\frac{C_{t+1}}{C_t}\Big)^{-\gamma} R_{t+1}\Big]
$$

We can convert this conditional expectation into an unconditional expectation by multiplying the expression using an instrument $z_t \in \mathcal{I}_t$ (E.g. $R_t$ or $\frac{C_t}{C_{t-1}}$)

$$ 
z_t = \mathbb{E}_t \Big[\delta \Big( \frac{C_{t+1}}{C_t}\Big)^{-\gamma} R_{t+1} z_t\Big]
$$

$$ 
\mathbb{E}_t \Big[\delta \Big( \frac{C_{t+1}}{C_t}\Big)^{-\gamma}  R_{t+1} z_t - z_t\Big] =0 
$$

$$ 
\mathbb{E} \Big[\mathbb{E}_t \Big[\delta \Big( \frac{C_{t+1}}{C_t}\Big)^{-\gamma} R_{t+1} z_t - z_t\Big]\Big]
$$

Law of iterated expectations

$$
\mathbb{E} \Big[ \Big(\delta \Big(\frac{C_{t+1}}{C_t} \Big)^{-\gamma}R_{t+1}-1  \Big)z_t \Big]=0
$$

Since we have 2 parameters we will use 3 instruments to estimate the model $z_t = 1, z_t = \Big(\frac{C_t}{C_{t-1}}\Big), z_t = R_t$

The residual vector becomes

$$
g_T(\theta)=\frac{1}{T}\sum_{t=1}^T \begin{bmatrix} \Big(\delta \Big(\frac{C_{t+1}}{C_t} \Big)^{-\gamma}R_{t+1}-1  \Big) \\ \Big(\delta \Big(\frac{C_{t+1}}{C_t} \Big)^{-\gamma}R_{t+1}-1  \Big)R_t \\ \Big(\delta \Big(\frac{C_{t+1}}{C_t} \Big)^{-\gamma}R_{t+1}-1  \Big) \frac{C_{t}}{C_{t-1}} \end{bmatrix}
$$

The GMM Estimator of $\delta$ and $\gamma$ comes from

$$
\hat{\theta} = \arg \min g_t(\theta)'Wg_t(\theta)
$$

## Obtain Data

## Data:

* Data on Returns from the US Market Portfolio
* Consumption data from the FRED

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import datetime
import urllib.request
import zipfile
from scipy.optimize import minimize
from scipy.stats import chi2

In [2]:
# Obtain Data
ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip"
# Download the file and save it
# We will name it fama_french.zip file
urllib.request.urlretrieve(ff_url,'fama_french.zip')
zip_file = zipfile.ZipFile('fama_french.zip', 'r')
# Next we extact the file data
# We will call it ff_factors.csv
zip_file.extractall()
# Make sure you close the file after extraction
zip_file.close()
ff_factors = pd.read_csv('F-F_Research_Data_Factors.csv', skiprows = 3, nrows = 1134)
ff_factors.tail()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
1129,202008,7.63,-0.22,-2.95,0.01
1130,202009,-3.63,0.04,-2.68,0.01
1131,202010,-2.10,4.36,4.21,0.01
1132,202011,12.47,5.82,2.14,0.01
1133,202012,4.63,4.89,-1.51,0.01


In [3]:
ff_factors["R"] = 1+(pd.to_numeric(ff_factors["Mkt-RF"]) + pd.to_numeric(ff_factors["RF"]))/100
ff_factors = ff_factors.rename(columns = {"Unnamed: 0" : "date"})
ff_factors = ff_factors[["date", "R"]]
ff_factors["date"] = pd.to_numeric(ff_factors["date"])
ff_factors.tail()

,date,R
1129,202008,1.0764
1130,202009,0.9638
1131,202010,0.9791
1132,202011,1.1248
1133,202012,1.0464


In [4]:
start = datetime.datetime (1920, 1, 1)
end = datetime.datetime (2020, 6, 1)

df = pdr.DataReader('PCE', 'fred', start, end)
df = df.rename(columns = {"PCE" : "C"})
df["date"] = pd.to_numeric(df.index.strftime("%Y%m"))
df = df.reset_index()
df = df[["date", "C"]]

In [5]:
df = pd.merge(ff_factors, df, on = ["date"])

In [6]:
def g_T(θ, return_covariance = False):
    δ = θ[0]
    γ = θ[1]
    C_tp1 = df["C"].shift(-1)
    C_t   = df["C"]
    C_tm1 = df["C"].shift(1)
    R_tp1 = df["R"].shift(-1)
    R_t   = df["R"]
    z_t = (1, R_t, C_t/C_tm1)
    u_T = np.zeros(len(z_t))

    v0 = (δ*((C_tp1/C_t)**-γ)*R_t-1)*1
    u_T[0] = np.nanmean(v0)
    v1 = (δ*((C_tp1/C_t)**-γ)*R_t-1)*R_t
    u_T[1] = np.nanmean(v1)
    v2 = (δ*((C_tp1/C_t)**-γ)*R_t-1)*C_t/C_tm1
    u_T[2] = np.nanmean(v2)
    
    G = np.column_stack((v0,v1,v2))
        
    if return_covariance:
        G= G[~np.isnan(G).any(axis=1)]
        return np.cov(G.T), G.shape[0]
    else:
        return u_T


    
    

In [17]:
def Q(θ, W=np.eye(3)):
    g = g_T(θ)
    return np.matmul(np.matmul(np.transpose(g),W),g)

In [26]:
θ0 = np.array([0.5, 20])
res = minimize(Q, θ0, tol=1e-12)

In [27]:
θ_hat = res.x
θ_hat # focus on gamma not delta, as the initial guess is important

array([ 1.06062032, 14.98628791])

In [28]:
# Second Step
S, T = g_T(θ_hat, return_covariance = True)
Q2 = lambda θ : Q(θ , W=np.linalg.inv(S))
res = minimize(Q2, θ0, tol=1e-12)

In [29]:
θ_hat = res.x
θ_hat

array([ 1.07392082, 17.18455612])

In [30]:
# Standard Errors
def Δg(θ):
    # Compute derivative using first difference
    dG1 = (g_T(θ+np.array([θ[0]/100,0]))-g_T(θ))/(θ[0]/100)
    dG2 = (g_T(θ+np.array([0, θ[1]/100]))-g_T(θ))/(θ[1]/100)
    return np.array([dG1, dG2]).T

In [34]:
D = (Δg(θ_hat))

Σ = np.matmul(np.matmul(D.T, np.linalg.inv(S)),D)*(1/T)
t_θ = θ_hat[0]/np.sqrt(Σ[0,0])
t_γ = (θ_hat[1])/np.sqrt(Σ[1,1])
t_γ

9687.495705174373

## Test of overidentifying restrictions

$$
    T g_T(\theta)'S^{-1}g_T(\theta) \sim \chi^2_{N-K}
$$

In [35]:
S, T= g_T(θ_hat, return_covariance = True)
X2 = T*Q(θ_hat, W = np.linalg.inv(S))
chi2.cdf(X2, 3-2, loc=0, scale=1)

0.6700593612674755